## 1. Basic API 

In [ ]:
from openai import OpenAI
client = OpenAI()

MAIN_CFG = {
    'model':"gpt-4o-mini", # can be 'gpt-4o', 'o1-mini'        
    'messages':[
        {"role": "user", "content": "explain in 300 words what's fine-tune in llm?"}
    ],
    'max_tokens':400,
    'temperature':1
}

completion = client.chat.completions.create(
    **MAIN_CFG
)

# completion.to_dict()['choices'][0]['message']['content'] # get content

## 2. Prompt
### prompt: instruction

In [ ]:
MAIN_CFG['messages'] += [{ # system instructions
      "role": "system",
      "content": [{
          "type": "text",
          "text": "you are a middle school teacher"
        }]
    }]
completion = client.chat.completions.create(
    **MAIN_CFG
)

### prompt: multi-turn conversation

In [ ]:
MAIN_CFG['messages'] = [
    {
      "role": "user", # usr-1
      "content": [{ "type": "text", "text": "knock knock." }]
    },
    {
      "role": "assistant", # reply-1
      "content": [{ "type": "text", "text": "Who's there?" }]
    },
    {
      "role": "user", # usr-2
      "content": [{ "type": "text", "text": "Orange." }]
    },
    {
      "role": "assistant", # reply-2
      "content": [{ "type": "text", "text": "Orange Who?" }]
    },
    {
      "role": "assistant", # usr-3
      "content": [{ "type": "text", "text": "Orange Orange." }]
    },
  ]

### prompt: parse docx to csv

In [ ]:
from docx import Document
def read_docx(file_path):
    doc = Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        text.append(paragraph.text)
    return '\n'.join(text)
    
MAIN_CFG['messages'] = [
    {
        'role':'system',
        'content':'You will be provided with unstructured data, and your task is to parse it into CSV format.'},
    {
        'role':'user',
        'content':read_docx('../../Data/1.docx')},
]


### prompt: choose in predefined list

In [ ]:
MAIN_CFG['messages'] = [
    {
        'role':'system',
        'content':'''
            You are to answer questions based on the presented content.
            Provide your answer in bullet point form.
            Answer ONLY based on the following list and choose either yes or no:
    - If contains budget plan for the project
    - Writes clearly about the objectives and project plan  
    - Writes neatly or not'''
    },
    {
        'role':'user',
        'content':read_docx('../../Data/2.docx')
    },
]

## 3. Image process

In [ ]:
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
import numpy as np

url = 'https://upload.wikimedia.org/wikipedia/en/thumb/3/34/Cars_2006.jpg/220px-Cars_2006.jpg'

im = np.array(Image.open(BytesIO(requests.get(url).content)))
plt.imshow(im)

In [ ]:
MAIN_CFG['messages'] = [
    {
        'role':'user',
        'content':[
            {"type": "text", "text": "Count cars by color"},
             {
              "type": "image_url",
              "image_url": {
                "url": url,
                "detail": "high" # high for more details(in counting), low for main description. 
             }}
        ],
    },
    {
        'role':'system',
        'content':'you are a research helper and answer precisely'
    }
]

In [ ]:
MAIN_CFG['max_tokens'] = 1024
MAIN_CFG['model'] = 'gpt-4o'
completion = client.chat.completions.create(
    **MAIN_CFG
)
completion.to_dict()['choices'][0]['message']['content']

In [ ]:
import base64
len(base64.b64encode(b'12345678abcdefgh').decode('utf-8')) # 16 bytes to 24 ascii bytes